In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
## Imports and environment variables 
import os
import torch


from travis_attack.utils import set_seed, set_session_options, prepare_logger
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from accelerate import Accelerator, notebook_launcher





In [ ]:
accelerator = Accelerator()
cfg = Config()
cfg.device = accelerator.device
cfg = cfg.adjust_config_for_simple_dataset()  # for testing, easier 
set_seed(cfg.seed)
set_session_options()
logger = prepare_logger()
vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model)
vm_model,pp_model,sts_model,optimizer,ds.dld_tkn['train'] = accelerator.prepare(vm_model,pp_model,sts_model,optimizer,ds.dld_tkn['train'])
cfg.n_train_steps = cfg.n_train_epochs * len(ds.dld_tkn['train'])



Using custom data configuration default-b253756c445fb811
Reusing dataset csv (/data/tproth/.cache/huggingface/datasets/csv/default-b253756c445fb811/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-c802946231f72062
Reusing dataset csv (/data/tproth/.cache/huggingface/datasets/csv/default-c802946231f72062/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-43a49c5188c42e69
Reusing dataset csv (/data/tproth/.cache/huggingface/datasets/csv/default-43a49c5188c42e69/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vars(cfg)

{'pp_name': 'eugenesiow/bart-paraphrase',
 'vm_name': 'textattack/distilbert-base-uncased-rotten-tomatoes',
 'sts_name': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
 'dataset_name': 'simple',
 'seed': 420,
 'use_fp16': True,
 'lr': 1e-05,
 'normalise_rewards': False,
 'metrics': ['loss',
  'pp_logp',
  'reward',
  'vm_score',
  'sts_score',
  'label_flip'],
 'pin_memory': True,
 'zero_grad_with_none': False,
 'pad_token_embeddings': True,
 'embedding_padding_multiple': 8,
 'orig_padding_multiple': 8,
 'bucket_by_length': True,
 'shuffle_train': False,
 'remove_misclassified_examples': True,
 'pp': {'num_beams': 1,
  'num_return_sequences': 1,
  'num_beam_groups': 1,
  'diversity_penalty': 0.0,
  'temperature': 1.5,
  'length_penalty': 1,
  'min_length': 5,
  'max_length': 20},
 'use_small_ds': False,
 'n_shards': None,
 'shard_contiguous': None,
 'save_model_while_training': False,
 'save_model_freq': 10,
 'wandb': {'mode': 'online',
  'log_grads': False,
  'log_grad